In [1]:
import numpy as np
import pandas as pd

from scipy import stats
from scipy.stats import chi2_contingency
from scipy.stats import ttest_ind

from pathlib import Path

In [2]:
BASE_DIR = Path(__name__).resolve().parent.parent
FILES_DIR = BASE_DIR / 'files'

In [3]:
bidding_origin = pd.read_csv(FILES_DIR / 'bidding_results.csv')
# bidding = pd.read_csv(FILES_DIR / 'bidding_results.csv')

In [5]:
# bidding = bidding.drop_duplicates(subset=['index', 'date'])
indexs = list(set(bidding_origin['index'].to_list()))
bidding = bidding_origin.drop_duplicates(['index'])

In [6]:
bidding

,index,Impression,Click,Cost,Sum of AD rank,Bid,Price(PC),Price(Mobile),Delivery Fee,Category Name 1,Category Name 2,Category Name 3,Category Name 4,date
0,3575092,3,0,0.0,3.000000,50,165900,165900,0,가구/인테리어,침실가구,장롱/붙박이장,드레스룸,20221031
1,3575637,6,0,0.0,3.333333,50,10900,10900,0,가구/인테리어,인테리어소품,아로마/캔들용품,아로마방향제/디퓨저,20221031
2,4129259,7,0,0.0,8.000000,50,255000,255000,-1,가구/인테리어,거실가구,소파,패브릭소파,20221031
3,4131479,36,3,150.0,4.416667,140,190000,190000,20000,가구/인테리어,아동/주니어가구,책상의자세트,NaN,20221031
4,4131482,224,0,0.0,5.696429,50,129000,129000,0,가구/인테리어,아동/주니어가구,책상,NaN,20221031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1337083,2566795,5,0,0.0,13.200000,500,19900,19900,0,가구/인테리어,인테리어소품,아로마/캔들용품,아로마방향제/디퓨저,20230103
1337084,2566880,3,0,0.0,17.000000,500,23900,23900,3000,생활/건강,문구/사무용품,이벤트/파티용품,데코용품,20230103
1337085,2566881,1,0,0.0,8.000000,500,2500,2500,3000,가구/인테리어,인테리어소품,기타장식용품,NaN,20230103
1337086,2566891,2,0,0.0,4.000000,500,21000,21000,3000,생활/건강,정원/원예용품,화분받침,NaN,20230103


#### 피어슨 상관 계수란??
두 변수의 선형 상관 관계를 계량화한 수치입니다.

결과값은 -1 ~ 1 사이의 값이며,

양의 상관 관계가 있을수록 1에 가깝고, 음의 상관 관계가 있을수록 -1에 가깝습니다.

또한, 상관 관계가 없을수록 0에 가깝습니다.

In [7]:
def pearson(a, b):
    pearson_r, p_value = stats.pearsonr(a, b)
    print("p 값 :", format(p_value, '.5f'))
    print("피어슨 상관계수 (r) :", format(pearson_r, '.5f'))

In [8]:
pearson(bidding['Impression'], bidding['Click'])

p 값 : 0.00000
피어슨 상관계수 (r) : 0.49195


In [9]:
bidding['Sum of AD rank'] = bidding['Sum of AD rank'].fillna(0) 

/var/folders/c1/gs670d5j53x4mqw97p1kg5r80000gn/T/ipykernel_20826/3880863545.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bidding['Sum of AD rank'] = bidding['Sum of AD rank'].fillna(0)


In [10]:
pearson(bidding['Sum of AD rank'], bidding['Click'])

p 값 : 0.00000
피어슨 상관계수 (r) : -0.09130


In [11]:
pearson(bidding['Bid'], bidding['Click'])

p 값 : 0.00000
피어슨 상관계수 (r) : 0.10907



#### t 검정 (t-test)
모집단의 분산이나 표준편차를 알지 못할 때  
모집단을 대표하는 표본으로부터 추정된 분산이나 표준편차를 가지고 검정하는 방법으로  
“두 모집단의 평균간의 차이는 없다”라는 귀무가설과 “두 모집단의 평균 간에 차이가 있다”라는 대립가설 중에 하나를 선택할 수 있도록 하는 통계적 검정방법이다.

In [59]:
t_score, p_value = ttest_ind(
    bidding[bidding['Impression'] > 10000]['Impression'],
    bidding[bidding['Impression'] < 10000]['Impression'],
    equal_var=False)

print("p 값 :", format(p_value, '.5f'))
print("t_score :", format(t_score, '.5f'))

p 값 : 0.00250
t_score : 4.59617


In [13]:

bidding["CTR"] = (bidding["Click"]/bidding["Impression"]) * 100
bidding["CPC"] = (bidding["Cost"]/bidding["Click"])
bidding["CPM"] = (bidding["Cost"]/1000)

/var/folders/c1/gs670d5j53x4mqw97p1kg5r80000gn/T/ipykernel_20826/3509760344.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bidding["CTR"] = (bidding["Click"]/bidding["Impression"]) * 100
/var/folders/c1/gs670d5j53x4mqw97p1kg5r80000gn/T/ipykernel_20826/3509760344.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bidding["CPC"] = (bidding["Cost"]/bidding["Click"])
/var/folders/c1/gs670d5j53x4mqw97p1kg5r80000gn/T/ipykernel_20826/3509760344.py:3: SettingWithCopyWarning: 
A value is trying to be set on a 

In [14]:
group_A = bidding[bidding['Impression'] < 10000]
group_B = bidding[bidding['Impression'] > 10000]

In [31]:
def click_abtest(a_click, total_a, b_click, total_b):
    click = [a_click, b_click]
    no_click = [total_a - a_click, total_b - b_click]
    cont_table = pd.DataFrame([click, no_click], columns=['A', 'B'], index=['click', 'no_click'])
    chi2, p_val, d_f, expected = chi2_contingency([click, no_click])

    print("카이제곱 통계량 :", format(chi2, '.5f'))
    print("p_value :", format(p_val, '.5f'))

In [64]:
# p_value 가 0.05보다 높으면 group_B가 더 낫다고 말할 수 없다
click_abtest(
    int(group_A[group_A['Click'] > 0]['index'].count()),
    int(group_A.shape[0]), 
    int(group_B[group_B['Click'] < 0]['index'].count()),
    int(group_B.shape[0]))

카이제곱 통계량 : 0.91015
p_value : 0.34007


In [50]:
for cate in categorys:
    df = bidding.loc[bidding['Category Name 1'] == cate]
    print(f"CTR : {cate}: {df['CTR'].mean()}")

CTR : 화장품/미용: 1.2473374933584835
CTR : 패션잡화: 1.6832246187605258
CTR : 디지털/가전: 1.9056306143950545
CTR : 출산/육아: 2.3344057771067575
CTR : 스포츠/레저: 2.4870991144796286
CTR : 식품: 0.0
CTR : 패션의류: 1.1409293403869583
CTR : 생활/건강: 1.7998665117753718
CTR : 여가/생활편의: 4.306358289042499
CTR : 가구/인테리어: 1.5326583146091546


In [46]:
categorys_df_dict = {}

for cate in categorys:
    df = bidding.loc[bidding['Category Name 1'] == cate]
    categorys_df_dict[cate] = df

In [53]:
group_a = categorys_df_dict['여가/생활편의']

In [60]:
for key in categorys_df_dict.keys():
    if key != '여가/생활편의':
        group_b = categorys_df_dict[key]
        
        print(key)
        
        t_score, p_value = ttest_ind(
            group_a['Impression'],
            group_b['Impression'],
            equal_var=False)

        print("p 값 :", format(p_value, '.5f'))
        print("t_score :", format(t_score, '.5f'))
        print('\n')

화장품/미용
p 값 : 0.26066
t_score : 1.14980


패션잡화
p 값 : 0.24810
t_score : 1.18124


디지털/가전
p 값 : 0.39020
t_score : 0.87304


출산/육아
p 값 : 0.35276
t_score : 0.94611


스포츠/레저
p 값 : 0.42846
t_score : 0.80326


식품
p 값 : 0.16522
t_score : 1.42749


패션의류
p 값 : 0.33997
t_score : 0.97144


생활/건강
p 값 : 0.26055
t_score : 1.15012


가구/인테리어
p 값 : 0.23165
t_score : 1.22473




In [52]:
for key in categorys_df_dict.keys():
    if key != '여가/생활편의':
        group_b = categorys_df_dict[key]
        
        print(key)
        
        click_abtest(
            int(group_a[group_a['Click'] > 0]['index'].count()),
            int(group_a.shape[0]), 
            int(group_b[group_b['Click'] > 0]['index'].count()),
            int(group_b.shape[0]))
        
        print('\n')

화장품/미용
카이제곱 통계량 : 0.00000
p_value : 1.00000


패션잡화
카이제곱 통계량 : 0.09139
p_value : 0.76242


디지털/가전
카이제곱 통계량 : 0.00000
p_value : 1.00000


출산/육아
카이제곱 통계량 : 0.68183
p_value : 0.40896


스포츠/레저
카이제곱 통계량 : 0.04439
p_value : 0.83312


식품
카이제곱 통계량 : 1.22541
p_value : 0.26830


패션의류
카이제곱 통계량 : 0.00000
p_value : 1.00000


생활/건강
카이제곱 통계량 : 0.06111
p_value : 0.80475


가구/인테리어
카이제곱 통계량 : 0.64400
p_value : 0.42227


